<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/notebooks/inference/TpuIndex_build_index_and_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Inference with TPU-Index

Download and install libraries, models, data, etc. 

In [0]:
!gdown --id "10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9"   # citation vectors
!gdown --id "1-23aNm7j0bnycvyd_OaQfofVYPTewgOI"   # abstract vectors
!gdown --id "1NyUQwgUNj9bFsiCnZ2TfKmWn5r-Y6wav"   # TitlesIdAbstractsEmbedIds
!wget 'https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar'
!tar -xvf 'scibert_scivocab_uncased.tar'

!pip install transformers --quiet

%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
from time import time
from tqdm import tqdm_notebook as tqdm
from transformers import BertTokenizer
import pandas as pd
from pprint import pprint
import psutil
import gc

print('TensorFlow:', tf.__version__)

Downloading...
From: https://drive.google.com/uc?id=10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9
To: /content/CitationSimilarityVectors106Epochs.npy
2.59GB [00:29, 87.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-23aNm7j0bnycvyd_OaQfofVYPTewgOI
To: /content/AbstractSimVectors.npy
2.59GB [00:44, 58.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NyUQwgUNj9bFsiCnZ2TfKmWn5r-Y6wav
To: /content/TitlesIdsAbstractsEmbedIdsCOMPLETE_12-30-19.json.gzip
432MB [00:05, 79.5MB/s]
--2020-01-06 04:58:56--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.200.160
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.200.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 421.96M  5

Install import tpu-index

In [0]:
!pip install tpu-index
from tpu_index import TPUIndex


INFO:tensorflow:Initializing the TPU system: 10.74.47.146:8470


INFO:tensorflow:Initializing the TPU system: 10.74.47.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Set up TPU-Index for the citation similarity vectors 

In [0]:
citations_embeddings = np.load('CitationSimilarityVectors106Epochs.npy')
indexC = TPUIndex(num_tpu_cores=8)
indexC.create_index(citations_embeddings.copy())
del citations_embeddings
print('System Memory used    : ' + str(psutil.virtual_memory().percent) + '%')


Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7
System Memory used    : 13.9%


# Set up TPU-Index for the abstract similarity vectors 

In [0]:
abstract_embeddings = np.load('AbstractSimVectors.npy')
indexA = TPUIndex(num_tpu_cores=8)
indexA.create_index(abstract_embeddings.copy())
del abstract_embeddings
print('System Memory used    : ' + str(psutil.virtual_memory().percent) + '%')

Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7
System Memory used    : 18.2%


Load model, tokenizer, and get semantic scholar papers data

In [0]:
model = tf.saved_model.load('gs://tfworld/saved_models')
tokenizer = BertTokenizer(vocab_file='scibert_scivocab_uncased/vocab.txt')

df = pd.read_json('/content/TitlesIdsAbstractsEmbedIdsCOMPLETE_12-30-19.json.gzip', compression = 'gzip')
embed2Title = pd.Series(df['title'].values,index=df['EmbeddingID']).to_dict()
embed2Abstract = pd.Series(df['paperAbstract'].values,index=df['EmbeddingID']).to_dict()
embed2Paper = pd.Series(df['id'].values,index=df['EmbeddingID']).to_dict()

Inference. Type in your query, and get results. 

### NOTE: The first search will take about 40 seconds for the TPU to warm up. Each subsequent search should only take 100-150 ms. 
---



In [0]:
query = "neural machine translation" #@param {type:"string"}

Num_Results_Per_search = 10 #@param {type:"integer"}

query_encoded = tokenizer.encode(query, max_length=512, pad_to_max_length=True)
query_encoded = tf.constant(query_encoded, dtype=tf.int32)[None, :]
print('\nAbstract : ')
pprint(query)

s = time()
bert_output = model(query_encoded)
xq = tf.nn.l2_normalize(bert_output, axis=1)
prediction_time = time() - s

aD, aI = indexA.search(xq, distance_metric='cosine', top_k=Num_Results_Per_search)
cD, cI = indexC.search(xq, distance_metric='cosine', top_k=Num_Results_Per_search)

print('Abstract Embedding Search')
print('*'*80)
for i in range(len(aI)):
    print('Title : ')
    pprint(embed2Title[aI[i]])
    print('\n')
    print('Abstract : ')
    pprint(embed2Abstract[aI[i]])
    print('\n')
    pprint('Link: www.semanticscholar.org/paper/'+embed2Paper[aI[i]])
    print('\n'*2)
    print('*'*80)
    print('\n')
print('\nNeighbours       :', aI )
print('Distances        :', np.round(aD, 4))

print('Citation Embedding Search')
for i in range(len(cI)):
    print('Title : ')
    pprint(embed2Title[cI[i]])
    print('\n')
    print('Abstract : ')
    pprint(embed2Abstract[cI[i]])
    print('\n')
    pprint('Link: www.semanticscholar.org/paper/'+embed2Paper[cI[i]])
    print('\n'*2)
    print('*'*80 )
    print('\n')
print('\nNeighbours       :', cI )
print('Distances        :', np.round(cD, 4))


Abstract : 
'neural machine translation'
Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7
Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU: